In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# 📐 Understanding SAR Geometrical Effects 📡✨  

We've learned that **radar measures time delays that can be convereted to radial distances**. Importantly, these are not distances between targets along the ground plane that we image. So, how do we **translate our range measurements (and resolution) to the ground?** 🤔  

---

## 📏 **From Radial (Slant Range) Resolution to Ground Range Resolution**  

When we achieve resolution due to the **chirp bandwidth**, it's in **radial distance** — also known as **slant range** in SAR terminology. But, the images we focus are projected onto the **ground plane**. So, how do we **connect our range resolution to the ground geometry?** 🤔  

### 🔑 **The Key Insight:**  
- 📡 Radar can measure the **distance** of targets based on the time delays of the echoes.  
- 🌍 Even when we use SAR techniques to **focus our data on the ground**, the **resolving power** remains in terms of **range (distance to target)** and **azimuth** (direction along the synthetic aperture).  
- 🔄 Our task? To **translate these measurements** into something more useful: the **geometry of the ground**.  

---

## 📡 **Why Side-Looking Geometry Matters**  

In **SAR imaging from aircraft or satellites**, we **always use a side-looking geometry**. But why? 🤔  

### **The Reasoning:**  
- 🔽 If we pointed the antenna **directly down (nadir)**, almost all echoes would arrive with the **same time delay**. 📏❌  
- 📉 This means we **couldn't separate echoes based on range**, resulting in a complete loss of resolution along the ground range axis.  

Remember our very first radar imaging examples a couple of notebooks back — the ones where we placed the radar on the ground and imaged the area in front of it. In those cases we naturally got a kind of **bird’s-eye view** of the scene, where distances along the ground translated neatly into different echo delays. 🐦👀  

✅ To **separate echoes in ground range**, we need to look **a bit away from nadir**. By tilting the antenna sideways, echoes from different points along the **ground range axis — the direction on the ground that’s perpendicular to the flight path** — arrive with **different time delays**, and voilà — we can tell them apart! ⏳✨  

---

## **How Ground Range Resolution Works** 🏞️

Imagine we have a **slant range resolution of 50 cm**, which we achieved with our **300 MHz chirp**. But what does this mean on the ground? 🤔  

- Two targets on the ground are **resolvable** if the difference in their **range paths** is at least the **slant range resolution**.  
- This separation in slant range can be converted to a separation on the ground using the **incidence angle**. _(the angle between the **vertical (nadir)** and the radar’s **line of sight**)_  

## 📏✨ The Tilted Ruler: Slant vs. Ground Range  

Radar doesn’t measure distance along the ground — it measures it **along its line of sight**.  It’s like trying to measure the floor with a **ruler held at an angle**. 🪜  

- From the radar’s point of view, the ruler’s ticks are **evenly spaced in slant range**. ✅  
- When you project those ticks onto the ground, the spacing changes:  
  - **Steep look (small incidence angle, near nadir):** each slant‑range tick spans a **longer** distance on the ground → **worse ground‑range resolution**.  
  - **Shallow/side look (large incidence angle):** each slant‑range tick spans a **shorter** distance on the ground → **better ground‑range resolution**.  

---

### 🌍 What does this mean for resolution?  
- **Looking straight down (nadir)**: the ruler is steep → few ticks fit into a given ground distance → the ground looks “chunky” → **poorer resolution**.  
- **Looking from the side (oblique view)**: the ruler is flatter → many ticks fit into the same ground distance → the ground looks “finer” → **better resolution**.  

👇 Below you’ll find a simulation where you can play with the **incidence angle** and watch the **tilted ruler effect** in action. 📏✨  

In [ ]:
def build_tilted_ruler_voila_app():
    """
    Voila-friendly interactive: Tilted Ruler (Slant vs. Ground Range)
    - Slant 'ruler' has evenly spaced ticks (what radar measures).
    - Each tick is projected to the ground along a line PERPENDICULAR to the slant.
    - Adjust incidence angle to see how ground tick spacing changes.
    - Uses ipywidgets.Output() to host the Matplotlib figure (so it's a proper Widget).
    - Returns a single top-level Widget you can display in Voila/Jupyter.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, Checkbox, VBox, HBox, HTML, Layout, Output
    from IPython.display import display, clear_output

    # --- Controls ---
    angle_slider = FloatSlider(
        description='Incidence (°)',
        min=5, max=80, step=1, value=40,
        readout=True, continuous_update=False, layout=Layout(width='250px')
    )
    angle_slider.style.description_width = 'initial'
    tick_spacing_slider = FloatSlider(
        description='Slant tick (m)',
        min=0.5, max=5.0, step=0.5, value=1.0,
        readout=True, continuous_update=False, layout=Layout(width='250px')
    )
    tick_spacing_slider.style.description_width = 'initial'
    ticks_count_slider = FloatSlider(
        description='Tick count',
        min=5, max=20, step=1, value=10,
        readout=True, continuous_update=False, layout=Layout(width='210px')
    )
    ticks_count_slider.style.description_width = 'initial'
    show_labels = Checkbox(
        description='Show ground tick labels',
        value=True, indent=False, layout=Layout(width='230px')
    )
    info = HTML()
    out = Output()  # hosts the figure

    def draw(*_):
        with out:
            clear_output(wait=True)

            theta = np.deg2rad(angle_slider.value)          # incidence from vertical
            ds = float(tick_spacing_slider.value)           # slant tick spacing (m)
            n = int(ticks_count_slider.value)               # number of ticks

            # Direction vectors
            v = np.array([np.sin(theta), np.cos(theta)])    # slant unit vector
            nvec = np.array([np.cos(theta), -np.sin(theta)]) # perpendicular unit vector

            # End of the slant ruler
            end_pt = n * ds * v
            x_end, y_end = end_pt

            # Slant tick positions
            s_vals = np.linspace(ds, n * ds, n)
            xs = s_vals * v[0]
            ys = s_vals * v[1]

            # Perpendicular projections to ground (y=0)
            t_vals = ys / np.sin(theta)
            xg = xs + t_vals * nvec[0]

            # Ground spacing
            mean_gs = np.mean(np.diff(xg)) if len(xg) >= 2 else None

            # Figure
            fig, ax = plt.subplots(figsize=(7, 5))

            ax.axhline(0, color='k', linewidth=1)  # ground line
            ax.plot([0, x_end], [0, y_end], linewidth=2)  # slant line
            ax.plot(xs, ys, 'o', markersize=4)  # slant ticks

            # Projection lines + footprints
            for k, (xi, yi, xgi) in enumerate(zip(xs, ys, xg), start=1):
                ax.plot([xi, xgi], [yi, 0], linestyle='--', linewidth=1, alpha=0.7)
                ax.plot([xgi], [0], marker='o', markersize=4)
                if show_labels.value:
                    ax.text(xgi, -0.06 * (abs(y_end) + 1.0), f'{k}',
                            ha='center', va='top', fontsize=8)

            # Annotations
            ax.text(x_end * 0.55, y_end * 0.55,
                    "Tilted ruler (slant)\nEqual tick marks",
                    fontsize=9, bbox=dict(boxstyle='round', alpha=0.08, pad=0.3))
            ax.text(min(x_end * 0.55, x_end + 0.1), 0.1,
                    "Ground line (tick projections)",
                    fontsize=9, bbox=dict(boxstyle='round', alpha=0.08, pad=0.3))

            # Framing
            xmax = max(xg.max() if len(xg) else 1.0, x_end)
            margin_x = max(1.0, 0.15 * (xmax + ds))
            margin_y = max(1.0, 0.20 * (y_end + ds))
            ax.set_xlim(-0.1 * (xmax + margin_x), xmax + margin_x)
            ax.set_ylim(-0.6 * margin_y, y_end + margin_y)
            ax.set_xlabel("Ground distance [m]")
            ax.set_ylabel("Height [m]")
            ax.set_title("Tilted Ruler: Perpendicular Projections to Ground")
            ax.grid(False)

            # Friendly narration
            if mean_gs:
                info.value = (
                    "<b>What to notice:</b> The ticks are evenly spaced along the <i>slant</i> ruler. "
                    "We drop lines <i>perpendicular</i> to that ruler to find where each tick lands on the ground. "
                    "When the view becomes more side-looking (ruler flatter), those ground footprints move closer together — the ground looks finer. "
                    "When the ruler is steeper (closer to straight down), their footprints spread out — the ground looks chunkier. "
                    f"<br><b>Current feel:</b> neighboring footprints are about <i>{mean_gs:.2f} m</i> apart on average."
                )
            else:
                info.value = (
                    "<b>What to notice:</b> The ticks are evenly spaced along the <i>slant</i> ruler. "
                    "We drop lines <i>perpendicular</i> to that ruler to find where each tick lands on the ground. "
                    "The ground footprints change spacing with angle."
                )

            plt.tight_layout()
            display(fig)
            plt.close(fig)

    # Wire controls
    for w in (angle_slider, tick_spacing_slider, ticks_count_slider, show_labels):
        w.observe(draw, names='value')

    # Initial render
    draw()

    controls = HBox([angle_slider, tick_spacing_slider, ticks_count_slider, show_labels])
    ui = VBox([controls, info, out])
    return ui

ruler_ui = build_tilted_ruler_voila_app()
display(ruler_ui)

### 🌌 **The Spaceborne SAR Trade-Off**  

Now, you might be wondering: 🤔  
**If low incidence angles give us bad ground range resolution, why don't we always use a high incidence angle?**  

Well, there are a couple of reasons:  

1. **Geometry Constraints:**  
   - With spaceborne SAR, our **orbit determines the geometry** when we pass over a region of interest. 🌍🛰️  
   - To **maximize the number of imaging opportunities** over a certain region, we might need some **flexibility in geometry**.  

2. **The Big Physical Reason:**  
   - When we **increase the incidence angle**, we are looking **further away in range**! 🌌📏  
   - And here’s where **physics comes to bite us**:  
     - We're already **struggling for power** because we are so far away from the ground.  
     - Increasing the incidence angle **makes our life even harder** in terms of **Signal-to-Noise Ratio (SNR)** due to the **radar range equation**.  

### ⚖️ **Balancing Act:**  
In small spaceborne SAR systems, we face a **trade-off:**  

- **Low Incidence Angle:**  
  - 🔋 Higher received signal power (**better SNR and NESZ**).  
  - 📉 Worse ground range resolution.  

- **High Incidence Angle:**  
  - 📸 Better ground range resolution.  
  - 📉 Weaker received signal power (**worse SNR and NESZ**) due to the **increased range**.  

✅ Finding the **right balance** between **resolution and signal power** is a critical part of SAR imaging. 🧩✨  

---

### Understanding Layover in SAR Imaging: Why Buildings and Mountains Lean Towards Us 🌄

Since SAR measures **distance (range) and position along the flight path (azimuth)**, every reflection, **no matter where it came from in 3D space**, gets mapped onto a **2D surface**.  What we end up with is a **2D representation of a 3D scene**, resulting in some fascinating **projection effects**!  

---

### 🔍📸 Optical Imaging vs. SAR Imaging: What's the Difference?

When we look at images produced by **optical cameras**, they are fundamentally different from **SAR images**. Let’s break down why:

---

### 📸 **How Optical Imaging Works:**  
- In **optical cameras**, the image plane is **perpendicular to the direction of incoming light rays**.  
- Light travels in straight lines, and the camera captures a **2D projection of the scene** from a single viewpoint.  
- This projection closely resembles how the human eye perceives the world. 🌍👁️  

---

### 📡 **How SAR Imaging Works:**  
- With **SAR**, we don’t have a typical camera lens. Instead, we use a **long, synthesized antenna** and measure the **distance to objects (range)**.  
- Instead of capturing light, SAR measures **microwave reflections**, and each pixel is determined by both **range (distance)** and **azimuth**.  
- What’s different from optical images? The **other dimension of our data** is actually along the **direction of illumination**, not perpendicular to it!

---

### ❌ **What's Missing?**  

SAR **can't determine the height** of targets directly — we can only separate targets based on **where they are along the line of sight and the flight track direction**. 📡👀  

Because of this, certain features of the scene may appear **distorted or misrepresented**, especially when dealing with **steep terrain or tall structures**. 🏔️🏢  

💡 **Here’s the problem:**  
Imagine we have a target that is not located on the **ground surface** where we want to reconstruct the image, but instead, at some height above it. Since SAR imaging **uses the distance to map points in the image**, this elevated point will appear **shifted** along the **ground range axis**. 📏 Specifically, its response will appear at the **ground range pixel** that has the **same distance (slant range)** as the elevated point **from the radar’s perspective**.  

---

### 🎟️👥 **The Ticket Line**  

Picture a line of people **queuing at a box office**.  
- The cashier is at the **same ground level** as the queue and follows one simple rule: **call whoever is closest in distance**. 🗣️  
- On the ground, “closest” just means the **first person in line**, so everything works as expected. ✅  

Now imagine the cashier’s desk is rebuilt at the **top of a staircase**, directly above its old spot. 🪜  
- The line stays on the ground. The **rule stays the same**: **call whoever is closest in straight‑line distance**.  
- From this higher vantage point, something odd happens: a **very tall person third in line** might have the **top of their head closer** to the cashier than the shorter people in front.  
- By the cashier’s rule, the tall person gets called first — even though, from the ground’s perspective, they’re further back. 🤯  

That’s **SAR layover** in a nutshell:  
- The radar “calls” pixels by **slant range** — the **straight‑line distance** along its line of sight.  
- So the **tops of tall objects** (mountains, buildings) can appear **ahead of their bases**, seeming to lean toward the radar in the final image.  

---

> 💡 **Side note – Foreshortening**  
> A closely related effect is **foreshortening**: instead of a single tall object tipping over, an **extended slope facing the radar** appears **compressed** in the image.  
> It’s the same geometry as layover, but stretched over a surface — so the effect is noticeable, yet usually less dramatic. ⛰️➡️📉  

👇 Explore layover with the interactive geometry below: adjust the **height of the radar** to change the incidence angle 📐, as well as the **building height** 🏙️. Watch how the **top** shifts toward the radar along slant range and can end up **projected ahead of its base** on the ground map — that’s **layover**. 

In [ ]:
def build_layover_groundmap_voila_app():
    """
    Voila-friendly illustration of SAR layover using true slant rays → ground-range mapping,
    with incidence angle (at the base) computed and optionally drawn.

    Shows:
      - Radar at (0, H_r) over flat ground (y=0).
      - A vertical object with base at x=xb and height H_obj.
      - Slant rays from radar to the object's base and top.
      - Equal-range circles (optional) centered at the radar for each slant distance.
      - Ground-range placements for base and top (where those ranges meet y=0).
      - Incidence angle at the base (angle between the incoming ray and the local vertical).
        * Numeric readout in the info panel.
        * Optional arc drawn at the base to visualize the angle.

    Returns an ipywidgets.VBox you can display in Jupyter/Voila.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, Checkbox, VBox, HBox, HTML, Layout, Output
    from IPython.display import display, clear_output

    # ---------- Controls ----------
    radar_h = FloatSlider(
        description='Radar height',
        min=50.0, max=200.0, step=1.0, value=60.0,
        continuous_update=False, readout=True, layout=Layout(width='240px')
    )
    radar_h.style.description_width = 'initial'
    base_x = FloatSlider(
        description='Base position',
        min=50.0, max=200.0, step=1.0, value=80.0,
        continuous_update=False, readout=True, layout=Layout(width='240px')
    )
    base_x.style.description_width = 'initial'
    obj_h = FloatSlider(
        description='Object height',
        min=10.0, max=60.0, step=1.0, value=25.0,
        continuous_update=False, readout=True, layout=Layout(width='240px')
    )
    obj_h.style.description_width = 'initial'
    show_circles = Checkbox(
        description='Show equal-range circles', value=True, indent=False,
        layout=Layout(width='220px')
    )
    show_incidence_arc = Checkbox(
        description='Show incidence arc', value=True, indent=False,
        layout=Layout(width='200px')
    )
    info = HTML()
    out = Output()

    # ---------- Helper: draw a (partial) equal-range circle ----------
    def _plot_circle(ax, center, radius, xmax):
        t = np.linspace(-np.pi/2, np.pi/2, 300)
        x = center[0] + radius * np.cos(t)
        y = center[1] + radius * np.sin(t)
        m = (x >= 0) & (x <= xmax*1.2) & (y >= 0)
        ax.plot(x[m], y[m], linestyle=':', linewidth=1)

    # ---------- Helper: draw incidence angle arc at base ----------
    def _draw_incidence_arc(ax, base_pt, radar_pt, arc_radius=12.0):
        """
        Draws the angle between local vertical (up from base) and the incoming ray (base→radar).
        """
        bx, by = base_pt
        vx, vy = 0.0, 1.0  # vertical up direction
        # Vector from base to radar
        rx, ry = radar_pt[0] - bx, radar_pt[1] - by

        # Angles in standard (x-right, y-up) coordinates
        ang_vert = np.arctan2(vy, vx)          # = pi/2
        ang_ray  = np.arctan2(ry, rx)

        # Ensure the arc spans the smaller angle between them
        # Normalize difference to [-pi, pi]
        d = (ang_ray - ang_vert + np.pi) % (2*np.pi) - np.pi
        t1, t2 = (ang_vert, ang_vert + d) if d >= 0 else (ang_ray, ang_ray - d)
        ts = np.linspace(t1, t2, 80)

        x_arc = bx + arc_radius * np.cos(ts)
        y_arc = by + arc_radius * np.sin(ts)
        ax.plot(x_arc, y_arc, linewidth=1.5)

        # Place a small label roughly in the middle of the arc
        mid = (t1 + t2) / 2.0
        ax.text(bx + (arc_radius + 6) * np.cos(mid),
                by + (arc_radius + 6) * np.sin(mid),
                "Incidence", fontsize=8, ha='center', va='center')

    # ---------- Main draw ----------
    def draw(*_):
        with out:
            clear_output(wait=True)

            Hr = float(radar_h.value)
            xb = float(base_x.value)
            H = float(obj_h.value)

            # Geometry (2D cross-section)
            radar = np.array([0.0, Hr])
            base = np.array([xb, 0.0])
            top  = np.array([xb, H])

            # Slant distances
            Rb = np.linalg.norm(base - radar)  # base slant
            Rt = np.linalg.norm(top  - radar)  # top slant

            # Incidence angle at base (angle between vertical and incoming ray)
            # cos(angle_from_vertical) = Hr / Rb  -> angle = arccos(Hr / Rb)
            if Rb > 0:
                inc_rad = np.arccos(Hr / Rb)
                inc_deg = float(np.degrees(inc_rad))
            else:
                inc_deg = float('nan')

            # Ground placements (where equal-range circle meets ground y=0)
            def ground_hit(R):
                val = R*R - Hr*Hr
                if val <= 0:
                    return None
                return np.sqrt(val)

            xg_base = ground_hit(Rb)
            xg_top  = ground_hit(Rt)

            # Figure and axes
            fig, ax = plt.subplots(figsize=(7.4, 5.2))

            # Ground line
            ax.axhline(0, color='k', linewidth=1)

            # Radar marker
            ax.plot(radar[0], radar[1], marker='^', markersize=9)
            ax.text(radar[0]+2, radar[1]+2, "Radar", fontsize=9)

            # Object
            ax.plot([base[0], top[0]], [base[1], top[1]], linewidth=3, label="Object")
            ax.plot(base[0], base[1], 'o')
            ax.plot(top[0],  top[1],  'o')
            ax.text(base[0]+5, 3, "Base", ha='center', va='top', fontsize=9)
            ax.text(top[0]+2, top[1], "Top", ha='left', va='bottom', fontsize=9)

            # Rays from radar to base and top
            ax.plot([radar[0], base[0]], [radar[1], base[1]], linestyle='--', linewidth=1.6, alpha=0.85)
            ax.plot([radar[0], top[0]],  [radar[1], top[1]],  linestyle='--', linewidth=1.6, alpha=0.85)

            # Equal-range circles (optional)
            xmax_guess = max(xb + 60.0, (xg_base or 0) + 40.0, (xg_top or 0) + 40.0)
            if show_circles.value:
                _plot_circle(ax, center=radar, radius=Rb, xmax=xmax_guess)
                _plot_circle(ax, center=radar, radius=Rt, xmax=xmax_guess)

            # Ground placements markers
            def draw_ground_marker(xg, label, y_offset=-4):
                if xg is None:
                    return
                ax.plot([xg, xg], [0, 4], linestyle=':', linewidth=1, alpha=0.9)  # stem
                ax.plot(xg, 0, marker='o', markersize=5)
                ax.text(xg, y_offset, label, ha='center', va='top', fontsize=9)

            draw_ground_marker(xg_base, "Base maps here", y_offset=-3)
            draw_ground_marker(xg_top,  "Top maps here",  y_offset=-5)

            # Optional incidence arc at base
            if show_incidence_arc.value:
                _draw_incidence_arc(ax, base, radar, arc_radius=12.0)

            # Verdict
            if (xg_base is not None) and (xg_top is not None):
                if xg_top < xg_base:
                    verdict = "➡️ <b>Layover:</b> the top maps in front of the base (leans toward radar)."
                else:
                    verdict = "✅ <b>No layover:</b> the base maps in front of the top."
            else:
                verdict = "ℹ️ Adjust sliders so both slant distances intersect the ground (increase base distance or lower radar height)."

            # Framing
            x_max = max(xmax_guess, xb + 50.0)
            y_max = max(H + 25.0, radar[1] + 12.0)
            ax.set_xlim(-10, x_max)
            ax.set_ylim(-15, y_max)
            ax.set_xlabel("Ground distance (ground-range axis)")
            ax.set_ylabel("Height")
            ax.set_title("Layover via Slant Rays → Ground-Range Mapping")
            ax.grid(False)

            # Info (with incidence angle readout)
            if np.isfinite(inc_deg):
                inc_text = f"{inc_deg:.1f}°"
            else:
                inc_text = "n/a"
            info.value = (
                "<b>What’s happening:</b> The radar measures distance along slanted rays to the base and the top. "
                "Each distance is placed on the ground where a point would have that same distance to the radar. "
                "If the top’s ground placement lands nearer than the base’s, the object appears to lean toward the radar — that’s <b>layover</b>. "
                f"<br><b>Incidence angle at base:</b> {inc_text} "
                f"<br><b>Current outcome:</b> {verdict}"
            )

            plt.tight_layout()
            display(fig)
            plt.close(fig)

    # Wire up events
    for w in (radar_h, base_x, obj_h, show_circles, show_incidence_arc):
        w.observe(draw, names='value')

    # Initial render
    draw()

    # ---- Two-row layout for controls ----
    row1 = HBox([radar_h, base_x, obj_h],
                layout=Layout(align_items='center'))
    row2 = HBox([show_circles, show_incidence_arc],
                layout=Layout(align_items='center'))

    controls = VBox([row1, row2])
    ui = VBox([controls, info, out])
    return ui


layover_ui = build_layover_groundmap_voila_app()
display(layover_ui)

## 🌑📡 Radar Shadows & Multipath Bounces  

### 🌑 **Radar Shadows: When the Beam Can’t See Behind**  
- Just like sunlight can’t reach behind a mountain ⛰️, radar waves can also be **blocked**.  
- If the terrain is very steep (or a tall building is in the way), the radar beam can’t “bend” around it.  
- The area behind the obstacle receives **no illumination** → it shows up as a **dark shadow** in the SAR image. 🕳️  
- The **shape and size** of the shadow depend on:  
  - **Incidence angle**: shallow look angles create **longer shadows**, steep angles cast shorter ones.  
  - **Terrain slope**: steeper slopes cast bigger shadows, gentle slopes hardly cast any.  

👉 Shadows are **empty information** — places where the radar simply couldn’t see.  

---

### 🔁 **Multipath / Double Bounce: When Echoes Take a Detour**  
- Sometimes radar waves don’t just bounce once… they take a little **extra trip**! ✨  
- For example, a wave may hit the **ground** and then a **building wall**, bouncing straight back to the radar.  
- Or it might bounce between two walls like in a narrow street (“urban canyon”). 🏢🏢  
- These **double reflections** arrive back at the radar with almost the **same strength as a direct reflection**.  
- In the SAR image, they often appear as **bright spots** or streaks, sometimes in places where no real object exists.  

👉 Multipath is like radar getting a story through an **echo chamber** — the information is real, but it took a more complicated route to get back.  

---

👇 In the simulation below, explore how **shadows** and **double‑bounce reflections** emerge in SAR geometry. Tweak the **radar height**, **object height**, and **ground position** to see the effects.

In [ ]:
def build_shadows_and_multipath_voila_app():
    """
    Voila-friendly schematic of radar shadows and multipath (double bounce).
    - Radar at left looking right.
    - An adjustable obstacle (height and slope).
    - Shows the radar line-of-sight and the shadow region behind the obstacle.
    - Optionally shows a simple multipath (ground -> wall -> radar) reflection.

    Returns a VBox widget with controls + figure.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import FloatSlider, Checkbox, VBox, HBox, HTML, Layout, Output
    from IPython.display import display, clear_output

    # -------- Controls --------
    radar_h = FloatSlider(description="Radar height", min=20, max=100, step=1, value=60,
                          layout=Layout(width="240px"))
    radar_h.style.description_width = 'initial'
    obs_h = FloatSlider(description="Obstacle height", min=10, max=80, step=1, value=40,
                        layout=Layout(width="240px"))
    obs_h.style.description_width = 'initial'
    obs_x = FloatSlider(description="Obstacle distance", min=50, max=150, step=2, value=90,
                        layout=Layout(width="240px"))
    obs_x.style.description_width = 'initial'
    show_doublebounce = Checkbox(description="Show double-bounce", value=True,
                                 layout=Layout(width="220px"))
    info = HTML()
    out = Output()

    def draw(*_):
        with out:
            clear_output(wait=True)

            Hr = radar_h.value
            Ho = obs_h.value
            Xo = obs_x.value

            radar = np.array([0.0, Hr])
            obs_base = np.array([Xo, 0.0])
            obs_top = np.array([Xo, Ho])

            # Slant line from radar to top of obstacle
            dx = Xo
            dy = Ho - Hr
            slope = dy / dx
            # Extend line of sight beyond obstacle
            x_far = Xo + 80
            y_far = Hr + slope * (x_far - 0)

            # Compute where line of sight hits the ground again
            if slope < 0:
                x_shadow_end = Hr / -slope
                x_shadow_start = Xo
            else:
                x_shadow_start = Xo
                x_shadow_end = None

            fig, ax = plt.subplots(figsize=(7, 5))

            # Ground line
            ax.axhline(0, color="k")
            ax.text(0.95, 0.05, "Ground", transform=ax.transAxes, ha="right", fontsize=9)

            # Radar marker
            ax.plot(radar[0], radar[1], marker="^", markersize=9, color="blue")
            ax.text(radar[0]+2, radar[1]+2, "Radar", fontsize=9)

            # Obstacle (rectangle)
            ax.add_patch(plt.Rectangle((Xo-5, 0), 10, Ho, color="brown", alpha=0.6))
            ax.text(Xo, Ho+3, "Obstacle", ha="center", fontsize=9)

            # Line of sight
            ax.plot([radar[0], obs_top[0]], [radar[1], obs_top[1]], "r--", label="Line of sight")
            ax.plot([obs_top[0], x_far], [obs_top[1], y_far], "r--")

            # Shadow region
            if x_shadow_end is not None and x_shadow_end > x_shadow_start:
                ax.fill_between([x_shadow_start, x_shadow_end], 0, 15,
                                color="gray", alpha=0.3, label="Radar shadow")
                ax.text((x_shadow_start+x_shadow_end)/2, 8, "Shadow", ha="center", fontsize=9)

            # Optional double-bounce
            if show_doublebounce.value:
                wall_point = np.array([Xo, Ho*0.6])
                ground_point = np.array([Xo-15, 0])
                ax.plot([radar[0], ground_point[0]], [radar[1], ground_point[1]], "b-.", lw=1.6)
                ax.plot([ground_point[0], wall_point[0]], [ground_point[1], wall_point[1]], "b-.", lw=1.6)
                ax.plot([wall_point[0], radar[0]], [wall_point[1], radar[1]], "b-.", lw=1.6)
                ax.text(ground_point[0], -4, "Ground bounce", ha="center", fontsize=8)
                ax.text(wall_point[0]+2, wall_point[1], "Wall bounce", fontsize=8)

            # Framing
            ax.set_xlim(-10, Xo+100)
            ax.set_ylim(-10, Hr+40)
            ax.set_xlabel("Ground distance")
            ax.set_ylabel("Height")
            ax.set_title("Radar Shadows & Double-Bounce Schematic")
            ax.legend(loc="upper right", fontsize=8)
            ax.grid(False)

            # Info text
            shadow_msg = "➡️ Shadow behind obstacle: radar cannot see terrain there." \
                if x_shadow_end else "✅ No extended shadow (steep incidence)."
            bounce_msg = "🔁 Double-bounce path visible (ground → wall → radar)." \
                if show_doublebounce.value else ""
            info.value = f"<b>What to notice:</b><br>{shadow_msg}<br>{bounce_msg}"

            plt.tight_layout()
            display(fig)
            plt.close(fig)

    for w in (radar_h, obs_h, obs_x, show_doublebounce):
        w.observe(draw, names="value")

    draw()
    controls = HBox([radar_h, obs_h, obs_x, show_doublebounce])
    return VBox([controls, info, out])

shadow_multipath_app = build_shadows_and_multipath_voila_app()
display(shadow_multipath_app)